# Note detection in Sheet Image and WAV file:

1.   Process the Sheet Image and determine the notes to be played.

2.   Process the 'wav' file to transcribe into MIDI and get the notes played.

3. Compare the 2 list of Notes for anomaly.




In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [10]:
import warnings 
warnings.filterwarnings('ignore')

# Installing necessary libraries

In [2]:
pip install oemer

     |████████████████████████████████| 1.3 MB 12.6 MB/s 
     |████████████████████████████████| 22.3 MB 58.6 MB/s 
     |████████████████████████████████| 27.4 MB 1.5 MB/s 
     |████████████████████████████████| 49.9 MB 35 kB/s 
     |████████████████████████████████| 104.8 MB 115 kB/s 
     |████████████████████████████████| 15.7 MB 33.3 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-py

In [3]:
pip install pretty_midi

     |████████████████████████████████| 5.6 MB 10.2 MB/s 
     |████████████████████████████████| 51 kB 6.8 MB/s 
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-py3-none-any.whl size=5591955 sha256=cff91a27ec3614f50d21495751d2806794d6fe0310ce5ee525683e900ba05bd4
  Stored in directory: /root/.cache/pip/wheels/ad/74/7c/a06473ca8dcb63efb98c1e67667ce39d52100f837835ea18fa
Successfully built pretty-midi


In [ ]:
# !oemer /content/gdrive/MyDrive/oemerfiles/SilentNightBeginner_Zoom.PNG

In [ ]:
# import music21 as m12
# from music21 import *
# import pretty_midi

# def xml_to_list(fn):
#   xml_data = m12.converter.parse(fn)
#   notes = []
#   for part in xml_data.parts:
#       instrument = part.getInstrument().instrumentName
#       #for note in part.recurse().notes:
#       notes_to_parse = part.recurse()
#       #finding whether a particular element is note or a chord
#       for element in notes_to_parse:  
#             #note`
#         if isinstance(element, note.Note):
#             notes.append(str(element.pitch))      
#   return notes

# fn = "SilentNightBeginner_Zoom.PNG.musicxml"

# sheetmusicnotes  = xml_to_list(fn)

# Transcription using Magenta

In [ ]:
# Copyright 2021 Google LLC. All Rights Reserved.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#     http://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

#@title Setup Environment
#@markdown Install MT3 and its dependencies (may take a few minutes).

!apt-get update -qq && apt-get install -qq libfluidsynth1 build-essential libasound2-dev libjack-dev

!pip install --upgrade pip
!pip install --upgrade jaxlib -f https://storage.googleapis.com/jax-releases/jax_releases.html
!pip install nest-asyncio
!pip install pyfluidsynth

# install t5x
!git clone --branch=main https://github.com/google-research/t5x
!mv t5x t5x_tmp; mv t5x_tmp/* .; rm -r t5x_tmp
!sed -i 's:jax\[tpu\]:jax:' setup.py
!python3 -m pip install -e .

# install mt3
!git clone --branch=main https://github.com/magenta/mt3
!mv mt3 mt3_tmp; mv mt3_tmp/* .; rm -r mt3_tmp
!python3 -m pip install -e .

# copy checkpoints
!gsutil -q -m cp -r gs://mt3/checkpoints .

# copy soundfont (originally from https://sites.google.com/site/soundfonts4u)
!gsutil -q -m cp gs://magentadata/soundfonts/SGM-v2.01-Sal-Guit-Bass-V1.3.sf2 

In [ ]:
#@title Imports and Definitions

import functools
import os

import numpy as np
import tensorflow.compat.v2 as tf

import functools
import gin
import jax
import librosa
import note_seq
import seqio
import t5
import t5x

from mt3 import metrics_utils
from mt3 import models
from mt3 import network
from mt3 import note_sequences
from mt3 import preprocessors
from mt3 import spectrograms
from mt3 import vocabularies

from google.colab import files

import nest_asyncio
nest_asyncio.apply()

SAMPLE_RATE = 16000
SF2_PATH = 'SGM-v2.01-Sal-Guit-Bass-V1.3.sf2'

def upload_audio(sample_rate):
  data = list(files.upload().values())
  if len(data) > 1:
    print('Multiple files uploaded; using only one.')
  return note_seq.audio_io.wav_data_to_samples_librosa(
    data[0], sample_rate=sample_rate)

def read_audio(wav_file,sample_rate):
  with open(wav_file, 'rb') as f:
    contents = f.read()
    return note_seq.audio_io.wav_data_to_samples_librosa(
    contents, sample_rate=sample_rate
  )


class InferenceModel(object):
  """Wrapper of T5X model for music transcription."""

  def __init__(self, checkpoint_path, model_type='mt3'):

    # Model Constants.
    if model_type == 'ismir2021':
      num_velocity_bins = 127
      self.encoding_spec = note_sequences.NoteEncodingSpec
      self.inputs_length = 512
    elif model_type == 'mt3':
      num_velocity_bins = 1
      self.encoding_spec = note_sequences.NoteEncodingWithTiesSpec
      self.inputs_length = 256
    else:
      raise ValueError('unknown model_type: %s' % model_type)

    gin_files = ['/content/mt3/gin/model.gin',
                 f'/content/mt3/gin/{model_type}.gin']

    self.batch_size = 8
    self.outputs_length = 1024
    self.sequence_length = {'inputs': self.inputs_length, 
                            'targets': self.outputs_length}

    self.partitioner = t5x.partitioning.PjitPartitioner(
        num_partitions=1)

    # Build Codecs and Vocabularies.
    self.spectrogram_config = spectrograms.SpectrogramConfig()
    self.codec = vocabularies.build_codec(
        vocab_config=vocabularies.VocabularyConfig(
            num_velocity_bins=num_velocity_bins))
    self.vocabulary = vocabularies.vocabulary_from_codec(self.codec)
    self.output_features = {
        'inputs': seqio.ContinuousFeature(dtype=tf.float32, rank=2),
        'targets': seqio.Feature(vocabulary=self.vocabulary),
    }

    # Create a T5X model.
    self._parse_gin(gin_files)
    self.model = self._load_model()

    # Restore from checkpoint.
    self.restore_from_checkpoint(checkpoint_path)

  @property
  def input_shapes(self):
    return {
          'encoder_input_tokens': (self.batch_size, self.inputs_length),
          'decoder_input_tokens': (self.batch_size, self.outputs_length)
    }

  def _parse_gin(self, gin_files):
    """Parse gin files used to train the model."""
    gin_bindings = [
        'from __gin__ import dynamic_registration',
        'from mt3 import vocabularies',
        'VOCAB_CONFIG=@vocabularies.VocabularyConfig()',
        'vocabularies.VocabularyConfig.num_velocity_bins=%NUM_VELOCITY_BINS'
    ]
    with gin.unlock_config():
      gin.parse_config_files_and_bindings(
          gin_files, gin_bindings, finalize_config=False)

  def _load_model(self):
    """Load up a T5X `Model` after parsing training gin config."""
    model_config = gin.get_configurable(network.T5Config)()
    module = network.Transformer(config=model_config)
    return models.ContinuousInputsEncoderDecoderModel(
        module=module,
        input_vocabulary=self.output_features['inputs'].vocabulary,
        output_vocabulary=self.output_features['targets'].vocabulary,
        optimizer_def=t5x.adafactor.Adafactor(decay_rate=0.8, step_offset=0),
        input_depth=spectrograms.input_depth(self.spectrogram_config))


  def restore_from_checkpoint(self, checkpoint_path):
    """Restore training state from checkpoint, resets self._predict_fn()."""
    train_state_initializer = t5x.utils.TrainStateInitializer(
      optimizer_def=self.model.optimizer_def,
      init_fn=self.model.get_initial_variables,
      input_shapes=self.input_shapes,
      partitioner=self.partitioner)

    restore_checkpoint_cfg = t5x.utils.RestoreCheckpointConfig(
        path=checkpoint_path, mode='specific', dtype='float32')

    train_state_axes = train_state_initializer.train_state_axes
    self._predict_fn = self._get_predict_fn(train_state_axes)
    self._train_state = train_state_initializer.from_checkpoint_or_scratch(
        [restore_checkpoint_cfg], init_rng=jax.random.PRNGKey(0))

  @functools.lru_cache()
  def _get_predict_fn(self, train_state_axes):
    """Generate a partitioned prediction function for decoding."""
    def partial_predict_fn(params, batch, decode_rng):
      return self.model.predict_batch_with_aux(
          params, batch, decoder_params={'decode_rng': None})
    return self.partitioner.partition(
        partial_predict_fn,
        in_axis_resources=(
            train_state_axes.params,
            t5x.partitioning.PartitionSpec('data',), None),
        out_axis_resources=t5x.partitioning.PartitionSpec('data',)
    )

  def predict_tokens(self, batch, seed=0):
    """Predict tokens from preprocessed dataset batch."""
    prediction, _ = self._predict_fn(
        self._train_state.params, batch, jax.random.PRNGKey(seed))
    return self.vocabulary.decode_tf(prediction).numpy()

  def __call__(self, audio):
    """Infer note sequence from audio samples.
    
    Args:
      audio: 1-d numpy array of audio samples (16kHz) for a single example.

    Returns:
      A note_sequence of the transcribed audio.
    """
    ds = self.audio_to_dataset(audio)
    ds = self.preprocess(ds)

    model_ds = self.model.FEATURE_CONVERTER_CLS(pack=False)(
        ds, task_feature_lengths=self.sequence_length)
    model_ds = model_ds.batch(self.batch_size)

    inferences = (tokens for batch in model_ds.as_numpy_iterator()
                  for tokens in self.predict_tokens(batch))

    predictions = []
    for example, tokens in zip(ds.as_numpy_iterator(), inferences):
      predictions.append(self.postprocess(tokens, example))

    result = metrics_utils.event_predictions_to_ns(
        predictions, codec=self.codec, encoding_spec=self.encoding_spec)
    return result['est_ns']

  def audio_to_dataset(self, audio):
    """Create a TF Dataset of spectrograms from input audio."""
    frames, frame_times = self._audio_to_frames(audio)
    return tf.data.Dataset.from_tensors({
        'inputs': frames,
        'input_times': frame_times,
    })

  def _audio_to_frames(self, audio):
    """Compute spectrogram frames from audio."""
    frame_size = self.spectrogram_config.hop_width
    padding = [0, frame_size - len(audio) % frame_size]
    audio = np.pad(audio, padding, mode='constant')
    frames = spectrograms.split_audio(audio, self.spectrogram_config)
    num_frames = len(audio) // frame_size
    times = np.arange(num_frames) / self.spectrogram_config.frames_per_second
    return frames, times

  def preprocess(self, ds):
    pp_chain = [
        functools.partial(
            t5.data.preprocessors.split_tokens_to_inputs_length,
            sequence_length=self.sequence_length,
            output_features=self.output_features,
            feature_key='inputs',
            additional_feature_keys=['input_times']),
        # Cache occurs here during training.
        preprocessors.add_dummy_targets,
        functools.partial(
            preprocessors.compute_spectrograms,
            spectrogram_config=self.spectrogram_config)
    ]
    for pp in pp_chain:
      ds = pp(ds)
    return ds

  def postprocess(self, tokens, example):
    tokens = self._trim_eos(tokens)
    start_time = example['input_times'][0]
    # Round down to nearest symbolic token step.
    start_time -= start_time % (1 / self.codec.steps_per_second)
    return {
        'est_tokens': tokens,
        'start_time': start_time,
        # Internal MT3 code expects raw inputs, not used here.
        'raw_inputs': []
    }

  @staticmethod
  def _trim_eos(tokens):
    tokens = np.array(tokens, np.int32)
    if vocabularies.DECODED_EOS_ID in tokens:
      tokens = tokens[:np.argmax(tokens == vocabularies.DECODED_EOS_ID)]
    return tokens

In [ ]:
#@title Load Model
#@markdown The `ismir2021` model transcribes piano only, with note velocities.
#@markdown The `mt3` model transcribes multiple simultaneous instruments,
#@markdown but without velocities.

MODEL = "mt3" #@param["ismir2021", "mt3"]

checkpoint_path = f'/content/checkpoints/{MODEL}/'

inference_model = InferenceModel(checkpoint_path, MODEL)

# Main Code


1.   Loop over the 'wav' file present in a directory.
2.   For each of the 'wav' file:
    1. Transcribe the 'wav' file to 'midi' file.
    2. Find the corresponding 'png' file, use 'omr' to produce 'musicxml'file.
    3. Use 'music21' to parse the 'midi' file and 'musicxml' file.
    4. Calculate the accuracy by comparing the notes out of the 2.
3. Take Mean of accuracy.



In [15]:
from glob import glob
import pandas as pd
import pretty_midi
import os
import numpy as np
from collections import Counter
from sklearn.metrics import accuracy_score
accuracy_list= []
import music21 as m12
from music21 import *

def load_sheet_notes(f,dest_dir):
  gtruth_notes = []
  print(f'file is {f}')
  try:
    !oemer -o {dest_dir} {f} --save-cache &
    search_string = dest_dir + get_file_name(f) + '*musicxml'
    print(f'Search string for png file is {search_string}')
    gtruth_file = glob(search_string)[0]
    return(xml_to_list(gtruth_file))
  except:
    return []


def xml_to_list(fn):
  xml_data = m12.converter.parse(fn)
  c = m12.converter.parse(fn)
#c.show('musicxml.pdf')
  c.write('midi',fp='/content/gdrive/MyDrive/musicxmlparser/a.mid')
  notes = load_notes('/content/gdrive/MyDrive/musicxmlparser/a.mid')
  return notes



def get_file_name(link):
    filename = link.split('/')[::-1][0]
    return filename.split('.')[::-1][1]

def get_file_extension(link):
    filename = link.split('/')[::-1][0]
    return filename.split('.')[::-1][0]

def load_notes(filename_midi):
  notes=[]
  midi_data = pretty_midi.PrettyMIDI(filename_midi)
  for instrument in midi_data.instruments:
    if not instrument.is_drum:
      for note in instrument.notes:
        notes.append(str(pretty_midi.note_number_to_name(note.pitch)))
  return notes

def calc_accuracy(df_dict):
  list_gtruth=[]
  list_transcribed=[]

  for i in df_dict.index:
    if (df_dict['Transcribed'][i].astype(int) > df_dict['GTruth'][i].astype(int)):
      for j in range(df_dict['Transcribed'][i].astype(int)):
        list_transcribed.append(i)
        if(j < df_dict['GTruth'][i].astype(int)):
          list_gtruth.append(i)
        else:
          list_gtruth.append('NL')
    else:
      for k in range(df_dict['GTruth'][i].astype(int)):
        list_gtruth.append(i)
        if(k < df_dict['Transcribed'][i].astype(int)):
          list_transcribed.append(i)
        else:
          list_transcribed.append('NL')

  return (accuracy_score(list_gtruth , list_transcribed)*100)

def confusion_matrix(df_dict):
  from sklearn.metrics import confusion_matrix
  from sklearn.utils.multiclass import unique_labels
  import seaborn as sns
  import matplotlib.pyplot as plt

  list_gtruth=[]
  list_transcribed=[]

  for i in df_dict.index:
    if (df_dict['Transcribed'][i].astype(int) > df_dict['GTruth'][i].astype(int)):
      for j in range(df_dict['Transcribed'][i].astype(int)):
        list_transcribed.append(i)
        if(j < df_dict['GTruth'][i].astype(int)):
          list_gtruth.append(i)
        else:
          list_gtruth.append('NL')
    else:
      for k in range(df_dict['GTruth'][i].astype(int)):
        list_gtruth.append(i)
        if(k < df_dict['Transcribed'][i].astype(int)):
          list_transcribed.append(i)
        else:
          list_transcribed.append('NL')

  labels_plot = unique_labels(list_gtruth, list_transcribed)
  cf_matrix = confusion_matrix(list_gtruth,list_transcribed,labels=labels_plot)

  fig, ax = plt.subplots(figsize=(11,11))         # Sample figsize in inches
  ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues')

  ax.set_title('Seaborn Confusion Matrix with labels\n\n');
  ax.set_xlabel('\nPredicted Notes')
  ax.set_ylabel('Actual Notes ');


  ax.xaxis.set_ticklabels(labels_plot)
  ax.yaxis.set_ticklabels(labels_plot)


  ## Display the visualization of the Confusion Matrix.
  plt.show()

##################### MAIN FUNCTION ###############################
def main():
  !export LOG_LEVEL='critical'
  d = {0:'GTruth', 1:'Transcribed'}
  skip_transcription=0
  base_path = '/content/gdrive/MyDrive/BaselineExperiments/BaselineDataset'
  os.chdir(base_path)
  transcribed_path = os.path.join(os.getcwd(), 'TranscribedFiles')

  if os.path.exists(transcribed_path):
    skip_transcription = 1
  else:
    os.mkdir('TranscribedFiles') # 1st run

  # start the process 
  wav_path = '/content/gdrive/MyDrive/BaselineExperiments/BaselineDataset/WavFiles/'
  #wav_path = '/content/gdrive/MyDrive/BaselineExperiments/BaselineDataset/testWavFiles/'
  musicxml_path = '/content/gdrive/MyDrive/BaselineExperiments/BaselineDataset/MusicXMLs/'

  #read all the wav filenames
  rec_files=[i for i in os.listdir(wav_path) if i.endswith(".wav")]

  for f in rec_files:
    gtruth_arr=[]
    transcribed_arr=[]
    grtruth_dict={}
    transcribed_dict={}
    df_dict = pd.DataFrame()

    print(f'Processing {f}...')

    if skip_transcription == 0:                    # Do transcription
      # Reading the wav file by converting to bytes
      audio = read_audio(os.path.join(wav_path,f),sample_rate=SAMPLE_RATE)
      
      # Transcribe the audio file
      est_ns = inference_model(audio)
      transcribed_file = os.path.join(transcribed_path, get_file_name(f) + '_transcribed.mid')
      note_seq.sequence_proto_to_midi_file(est_ns,transcribed_file )
    else:
      print(f'Skipping the transcription for file {f}...')
      transcribed_file = os.path.join(transcribed_path, get_file_name(f) + '_transcribed.mid')


    # find corresponding Gtruth PNG file for comparison

    search_string = base_path + '/*/*/' + get_file_name(f) + '*png'
    print(f'Search string for png file is {search_string}')
    gtruth_file = glob(search_string)[0]
    print(f'Found corresponding PNG file {gtruth_file}')

    gtruth_arr = load_sheet_notes(gtruth_file,musicxml_path)
    transcribed_arr = load_notes(transcribed_file)

    # print(f'gtruth_arr is {gtruth_arr}')
    # print(f'transcribed_arr is {transcribed_arr}')
    if len(gtruth_arr) == 0:
      print(f'skipping song {f} because of OMR errors!')
      continue
    grtruth_dict = Counter(gtruth_arr)
    transcribed_dict = Counter(transcribed_arr)

    df_dict = pd.DataFrame([grtruth_dict,transcribed_dict]).transpose()
    df_dict.rename(columns = d, inplace = True) 

    acc = calc_accuracy(df_dict)
    print(f'Accuracy for song {f} is {acc:.2f}%' )

    accuracy_list.append(acc)

  # # Outside for loop

  print(f'Mean accuracy is {np.mean(accuracy_list):.2f}%')
  # print()
  # confusion_matrix(df_dict)


if __name__ == "__main__":
    main()

Processing OHolyNightInter.wav...
Skipping the transcription for file OHolyNightInter.wav...
Search string for png file is /content/gdrive/MyDrive/BaselineExperiments/BaselineDataset/*/*/OHolyNightInter*png
Found corresponding PNG file /content/gdrive/MyDrive/BaselineExperiments/BaselineDataset/SheetImages/PNGFiles/OHolyNightInter.png
file is /content/gdrive/MyDrive/BaselineExperiments/BaselineDataset/SheetImages/PNGFiles/OHolyNightInter.png
2022-03-11 11:03:56 Dewarping
2022-03-11 11:04:05 Extracting stafflines
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
27 219 16
220 412 16
413 605 16
606 798 16
799 991 16
992 1184 16
1185 1568 15
2022-03-11 11:04:07 Extracting noteheads
2022-03-11 11:04:07 Analyzing notehead bboxes
2022-03-11 11:04:09 Ins

musicxml.xmlToM21: WARNING: Cannot put in an element with a missing voice tag when no previous voice tag was given.  Assuming voice 1... 
musicxml.xmlToM21: WARNING: Cannot put in an element with a missing voice tag when no previous voice tag was given.  Assuming voice 1... 
musicxml.xmlToM21: WARNING: Cannot put in an element with a missing voice tag when no previous voice tag was given.  Assuming voice 1... 
musicxml.xmlToM21: WARNING: Cannot put in an element with a missing voice tag when no previous voice tag was given.  Assuming voice 1... 


Accuracy for song AveMariaPro.wav is 1.89%
Processing LittleDrummerBoyInter.wav...
Skipping the transcription for file LittleDrummerBoyInter.wav...
Search string for png file is /content/gdrive/MyDrive/BaselineExperiments/BaselineDataset/*/*/LittleDrummerBoyInter*png
Found corresponding PNG file /content/gdrive/MyDrive/BaselineExperiments/BaselineDataset/SheetImages/PNGFiles/LittleDrummerBoyInter.png
file is /content/gdrive/MyDrive/BaselineExperiments/BaselineDataset/SheetImages/PNGFiles/LittleDrummerBoyInter.png
2022-03-11 11:23:48 Extracting staffline and symbols
2022-03-11 11:23:48.318254590 [W:onnxruntime:Default, cuda_execution_provider.cc:2168 ConvTransposeNeedFallbackToCPU] Dropping the ConvTranspose node: ConvTranspose__19277 to CPU because it requires asymmetric padding which the CUDA EP currently does not support
2022-03-11 11:23:48.318312790 [W:onnxruntime:Default, cuda_execution_provider.cc:2277 GetCapability] CUDA kernel not supported. Fallback to CPU execution provider fo

musicxml.xmlToM21: WARNING: Cannot put in an element with a missing voice tag when no previous voice tag was given.  Assuming voice 1... 
musicxml.xmlToM21: WARNING: Cannot put in an element with a missing voice tag when no previous voice tag was given.  Assuming voice 1... 
musicxml.xmlToM21: WARNING: Cannot put in an element with a missing voice tag when no previous voice tag was given.  Assuming voice 1... 
musicxml.xmlToM21: WARNING: Cannot put in an element with a missing voice tag when no previous voice tag was given.  Assuming voice 1... 


Accuracy for song WaltzoftheflowersAdv.wav is 13.72%
Processing TheSoundsofSnowfallInter.wav...
Skipping the transcription for file TheSoundsofSnowfallInter.wav...
Search string for png file is /content/gdrive/MyDrive/BaselineExperiments/BaselineDataset/*/*/TheSoundsofSnowfallInter*png
Found corresponding PNG file /content/gdrive/MyDrive/BaselineExperiments/BaselineDataset/SheetImages/PNGFiles/TheSoundsofSnowfallInter.png
file is /content/gdrive/MyDrive/BaselineExperiments/BaselineDataset/SheetImages/PNGFiles/TheSoundsofSnowfallInter.png
2022-03-11 11:42:47 Extracting staffline and symbols
2022-03-11 11:42:47.408539993 [W:onnxruntime:Default, cuda_execution_provider.cc:2168 ConvTransposeNeedFallbackToCPU] Dropping the ConvTranspose node: ConvTranspose__19277 to CPU because it requires asymmetric padding which the CUDA EP currently does not support
2022-03-11 11:42:47.408586891 [W:onnxruntime:Default, cuda_execution_provider.cc:2277 GetCapability] CUDA kernel not supported. Fallback to 

...

Accuracy for song AllIwantforChristmasInter.wav is 23.79%
Processing RudolphtheRedInter.wav...
Skipping the transcription for file RudolphtheRedInter.wav...
Search string for png file is /content/gdrive/MyDrive/BaselineExperiments/BaselineDataset/*/*/RudolphtheRedInter*png
Found corresponding PNG file /content/gdrive/MyDrive/BaselineExperiments/BaselineDataset/SheetImages/PNGFiles/RudolphtheRedInter.png
file is /content/gdrive/MyDrive/BaselineExperiments/BaselineDataset/SheetImages/PNGFiles/RudolphtheRedInter.png
2022-03-11 13:01:30 Extracting staffline and symbols
2022-03-11 13:01:31.234523821 [W:onnxruntime:Default, cuda_execution_provider.cc:2168 ConvTransposeNeedFallbackToCPU] Dropping the ConvTranspose node: ConvTranspose__19277 to CPU because it requires asymmetric padding which the CUDA EP currently does not support
2022-03-11 13:01:31.234583904 [W:onnxruntime:Default, cuda_execution_provider.cc:2277 GetCapability] CUDA kernel not supported. Fallback to CPU execution provider fo

musicxml.xmlToM21: WARNING: Cannot put in an element with a missing voice tag when no previous voice tag was given.  Assuming voice 1... 


Accuracy for song JinglebellsInter.wav is 58.51%
Processing DanceofthePro.wav...
Skipping the transcription for file DanceofthePro.wav...
Search string for png file is /content/gdrive/MyDrive/BaselineExperiments/BaselineDataset/*/*/DanceofthePro*png
Found corresponding PNG file /content/gdrive/MyDrive/BaselineExperiments/BaselineDataset/SheetImages/PNGFiles/DanceofthePro.png
file is /content/gdrive/MyDrive/BaselineExperiments/BaselineDataset/SheetImages/PNGFiles/DanceofthePro.png
2022-03-11 13:11:49 Extracting staffline and symbols
2022-03-11 13:11:50.187437582 [W:onnxruntime:Default, cuda_execution_provider.cc:2168 ConvTransposeNeedFallbackToCPU] Dropping the ConvTranspose node: ConvTranspose__19277 to CPU because it requires asymmetric padding which the CUDA EP currently does not support
2022-03-11 13:11:50.187485340 [W:onnxruntime:Default, cuda_execution_provider.cc:2277 GetCapability] CUDA kernel not supported. Fallback to CPU execution provider for Op type: ConvTranspose node name

musicxml.xmlToM21: WARNING: Cannot put in an element with a missing voice tag when no previous voice tag was given.  Assuming voice 1... 
musicxml.xmlToM21: WARNING: Cannot put in an element with a missing voice tag when no previous voice tag was given.  Assuming voice 1... 
musicxml.xmlToM21: WARNING: Cannot put in an element with a missing voice tag when no previous voice tag was given.  Assuming voice 1... 
musicxml.xmlToM21: WARNING: Cannot put in an element with a missing voice tag when no previous voice tag was given.  Assuming voice 1... 
musicxml.xmlToM21: WARNING: Cannot put in an element with a missing voice tag when no previous voice tag was given.  Assuming voice 1... 
musicxml.xmlToM21: WARNING: Cannot put in an element with a missing voice tag when no previous voice tag was given.  Assuming voice 1... 
musicxml.xmlToM21: WARNING: Cannot put in an element with a missing voice tag when no previous voice tag was given.  Assuming voice 1... 
musicxml.xmlToM21: WARNING: Cannot

Accuracy for song JesuJoyPro.wav is 27.33%
Mean accuracy is 36.71%


# Main Code


1.   Loop over the 'wav' file present in a directory.
2.   For each of the 'wav' file:
    1. Transcribe the 'wav' file to 'midi' file.
    2. Find the corresponding 'png' file, use 'omr' to produce 'musicxml'file.
    3. Use 'music21' to parse the 'midi' file and convert the 'musicxml' to 'midi' file
    4. Calculate the accuracy by comparing the notes out of the 2.
3. Take Mean of accuracy.



In [16]:
from glob import glob
import pandas as pd
import pretty_midi
import os
import numpy as np
from collections import Counter
from sklearn.metrics import accuracy_score
accuracy_list= []
import music21 as m12
from music21 import *

def load_sheet_notes(f,dest_dir):
  gtruth_notes = []
  print(f'file is {f}')
  try:
    !oemer -o {dest_dir} {f} --save-cache &
    search_string = dest_dir + get_file_name(f) + '*musicxml'
    print(f'Search string for png file is {search_string}')
    gtruth_file = glob(search_string)[0]
    return(xml_to_list(gtruth_file))
  except:
    return []


def xml_to_list(fn):
  xml_data = m12.converter.parse(fn)
  c = m12.converter.parse(fn)
#c.show('musicxml.pdf')
  c.write('midi',fp='/content/gdrive/MyDrive/musicxmlparser/a.mid')
  notes = load_notes('/content/gdrive/MyDrive/musicxmlparser/a.mid')
  return notes



def get_file_name(link):
    filename = link.split('/')[::-1][0]
    return filename.split('.')[::-1][1]

def get_file_extension(link):
    filename = link.split('/')[::-1][0]
    return filename.split('.')[::-1][0]

def load_notes(filename_midi):
  notes=[]
  midi_data = pretty_midi.PrettyMIDI(filename_midi)
  for instrument in midi_data.instruments:
    if not instrument.is_drum:
      for note in instrument.notes:
        notes.append(str(pretty_midi.note_number_to_name(note.pitch)))
  return notes

def calc_accuracy(df_dict):
  list_gtruth=[]
  list_transcribed=[]

  for i in df_dict.index:
    if (df_dict['Transcribed'][i].astype(int) > df_dict['GTruth'][i].astype(int)):
      for j in range(df_dict['Transcribed'][i].astype(int)):
        list_transcribed.append(i)
        if(j < df_dict['GTruth'][i].astype(int)):
          list_gtruth.append(i)
        else:
          list_gtruth.append('NL')
    else:
      for k in range(df_dict['GTruth'][i].astype(int)):
        list_gtruth.append(i)
        if(k < df_dict['Transcribed'][i].astype(int)):
          list_transcribed.append(i)
        else:
          list_transcribed.append('NL')

  return (accuracy_score(list_gtruth , list_transcribed)*100)

def confusion_matrix(df_dict):
  from sklearn.metrics import confusion_matrix
  from sklearn.utils.multiclass import unique_labels
  import seaborn as sns
  import matplotlib.pyplot as plt

  list_gtruth=[]
  list_transcribed=[]

  for i in df_dict.index:
    if (df_dict['Transcribed'][i].astype(int) > df_dict['GTruth'][i].astype(int)):
      for j in range(df_dict['Transcribed'][i].astype(int)):
        list_transcribed.append(i)
        if(j < df_dict['GTruth'][i].astype(int)):
          list_gtruth.append(i)
        else:
          list_gtruth.append('NL')
    else:
      for k in range(df_dict['GTruth'][i].astype(int)):
        list_gtruth.append(i)
        if(k < df_dict['Transcribed'][i].astype(int)):
          list_transcribed.append(i)
        else:
          list_transcribed.append('NL')

  labels_plot = unique_labels(list_gtruth, list_transcribed)
  cf_matrix = confusion_matrix(list_gtruth,list_transcribed,labels=labels_plot)

  fig, ax = plt.subplots(figsize=(11,11))         # Sample figsize in inches
  ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues')

  ax.set_title('Seaborn Confusion Matrix with labels\n\n');
  ax.set_xlabel('\nPredicted Notes')
  ax.set_ylabel('Actual Notes ');


  ax.xaxis.set_ticklabels(labels_plot)
  ax.yaxis.set_ticklabels(labels_plot)


  ## Display the visualization of the Confusion Matrix.
  plt.show()

##################### MAIN FUNCTION ###############################
def main():
  !export LOG_LEVEL='critical'
  d = {0:'GTruth', 1:'Transcribed'}
  skip_transcription=0
  base_path = '/content/gdrive/MyDrive/BaselineExperiments/BaselineDataset'
  os.chdir(base_path)
  transcribed_path = os.path.join(os.getcwd(), 'TranscribedFiles')

  if os.path.exists(transcribed_path):
    skip_transcription = 1
  else:
    os.mkdir('TranscribedFiles') # 1st run

  # start the process 
  wav_path = '/content/gdrive/MyDrive/BaselineExperiments/BaselineDataset/WavFiles/'
  #wav_path = '/content/gdrive/MyDrive/BaselineExperiments/BaselineDataset/testWavFiles/'
  musicxml_path = '/content/gdrive/MyDrive/BaselineExperiments/BaselineDataset/MusicXMLs/'

  #read all the wav filenames
  rec_files=[i for i in os.listdir(wav_path) if i.endswith(".wav")]

  for f in rec_files:
    gtruth_arr=[]
    transcribed_arr=[]
    grtruth_dict={}
    transcribed_dict={}
    df_dict = pd.DataFrame()

    print(f'Processing {f}...')

    if skip_transcription == 0:                    # Do transcription
      # Reading the wav file by converting to bytes
      audio = read_audio(os.path.join(wav_path,f),sample_rate=SAMPLE_RATE)
      
      # Transcribe the audio file
      est_ns = inference_model(audio)
      transcribed_file = os.path.join(transcribed_path, get_file_name(f) + '_transcribed.mid')
      note_seq.sequence_proto_to_midi_file(est_ns,transcribed_file )
    else:
      print(f'Skipping the transcription for file {f}...')
      transcribed_file = os.path.join(transcribed_path, get_file_name(f) + '_transcribed.mid')


    # find corresponding Gtruth PNG file for comparison

    search_string = base_path + '/*/*/' + get_file_name(f) + '*png'
    print(f'Search string for png file is {search_string}')
    gtruth_file = glob(search_string)[0]
    print(f'Found corresponding PNG file {gtruth_file}')

    gtruth_arr = load_sheet_notes(gtruth_file,musicxml_path)
    transcribed_arr = load_notes(transcribed_file)

    # print(f'gtruth_arr is {gtruth_arr}')
    # print(f'transcribed_arr is {transcribed_arr}')
    if len(gtruth_arr) == 0:
      print(f'skipping song {f} because of OMR errors!')
      continue
    grtruth_dict = Counter(gtruth_arr)
    transcribed_dict = Counter(transcribed_arr)

    df_dict = pd.DataFrame([grtruth_dict,transcribed_dict]).transpose()
    df_dict.rename(columns = d, inplace = True) 

    acc = calc_accuracy(df_dict)
    print(f'Accuracy for song {f} is {acc:.2f}%' )

    accuracy_list.append(acc)

  # # Outside for loop

  print(f'Mean accuracy is {np.mean(accuracy_list):.2f}%')
  # print()
  # confusion_matrix(df_dict)


if __name__ == "__main__":
    main()

Processing OHolyNightInter.wav...
Skipping the transcription for file OHolyNightInter.wav...
Search string for png file is /content/gdrive/MyDrive/BaselineExperiments/BaselineDataset/*/*/OHolyNightInter*png
Found corresponding PNG file /content/gdrive/MyDrive/BaselineExperiments/BaselineDataset/SheetImages/PNGFiles/OHolyNightInter.png
file is /content/gdrive/MyDrive/BaselineExperiments/BaselineDataset/SheetImages/PNGFiles/OHolyNightInter.png
2022-03-11 13:37:59 Dewarping
2022-03-11 13:38:08 Extracting stafflines
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
27 219 16
220 412 16
413 605 16
606 798 16
799 991 16
992 1184 16
1185 1568 15
2022-03-11 13:38:10 Extracting noteheads
2022-03-11 13:38:10 Analyzing notehead bboxes
2022-03-11 13:38:12 Ins

musicxml.xmlToM21: WARNING: Cannot put in an element with a missing voice tag when no previous voice tag was given.  Assuming voice 1... 
musicxml.xmlToM21: WARNING: Cannot put in an element with a missing voice tag when no previous voice tag was given.  Assuming voice 1... 
musicxml.xmlToM21: WARNING: Cannot put in an element with a missing voice tag when no previous voice tag was given.  Assuming voice 1... 
musicxml.xmlToM21: WARNING: Cannot put in an element with a missing voice tag when no previous voice tag was given.  Assuming voice 1... 


Accuracy for song AveMariaPro.wav is 11.77%
Processing LittleDrummerBoyInter.wav...
Skipping the transcription for file LittleDrummerBoyInter.wav...
Search string for png file is /content/gdrive/MyDrive/BaselineExperiments/BaselineDataset/*/*/LittleDrummerBoyInter*png
Found corresponding PNG file /content/gdrive/MyDrive/BaselineExperiments/BaselineDataset/SheetImages/PNGFiles/LittleDrummerBoyInter.png
file is /content/gdrive/MyDrive/BaselineExperiments/BaselineDataset/SheetImages/PNGFiles/LittleDrummerBoyInter.png
2022-03-11 13:42:36 Dewarping
2022-03-11 13:42:43 Extracting stafflines
27 210 12
211 394 12
395 578 12
579 762 12
763 946 12
947 1130 12
1131 1496 10
2022-03-11 13:42:45 Extracting noteheads
2022-03-11 13:42:45 Analyzing notehead bboxes
2022-03-11 13:42:46 Instanitiating notes
2022-03-11 13:42:48 Grouping noteheads
2022-03-11 13:42:53 Extracting symbols
2022-03-11 13:42:54 Extracting rhythm types
2022-03-11 13:43:06 Building MusicXML document
Search string for png file is /c

musicxml.xmlToM21: WARNING: Cannot put in an element with a missing voice tag when no previous voice tag was given.  Assuming voice 1... 
musicxml.xmlToM21: WARNING: Cannot put in an element with a missing voice tag when no previous voice tag was given.  Assuming voice 1... 
musicxml.xmlToM21: WARNING: Cannot put in an element with a missing voice tag when no previous voice tag was given.  Assuming voice 1... 
musicxml.xmlToM21: WARNING: Cannot put in an element with a missing voice tag when no previous voice tag was given.  Assuming voice 1... 


Accuracy for song WaltzoftheflowersAdv.wav is 26.35%
Processing TheSoundsofSnowfallInter.wav...
Skipping the transcription for file TheSoundsofSnowfallInter.wav...
Search string for png file is /content/gdrive/MyDrive/BaselineExperiments/BaselineDataset/*/*/TheSoundsofSnowfallInter*png
Found corresponding PNG file /content/gdrive/MyDrive/BaselineExperiments/BaselineDataset/SheetImages/PNGFiles/TheSoundsofSnowfallInter.png
file is /content/gdrive/MyDrive/BaselineExperiments/BaselineDataset/SheetImages/PNGFiles/TheSoundsofSnowfallInter.png
2022-03-11 13:46:33 Dewarping
2022-03-11 13:46:39 Extracting stafflines
27 204 8
205 382 8
383 560 8
561 738 8
739 916 8
917 1094 8
1095 1272 8
1273 1454 8
2022-03-11 13:46:40 Extracting noteheads
2022-03-11 13:46:40 Analyzing notehead bboxes
2022-03-11 13:46:41 Instanitiating notes
2022-03-11 13:46:42 Grouping noteheads
2022-03-11 13:46:47 Extracting symbols
2022-03-11 13:46:47 Extracting rhythm types
2022-03-11 13:46:51 Building MusicXML document
Sea

...

Accuracy for song AllIwantforChristmasInter.wav is 55.64%
Processing RudolphtheRedInter.wav...
Skipping the transcription for file RudolphtheRedInter.wav...
Search string for png file is /content/gdrive/MyDrive/BaselineExperiments/BaselineDataset/*/*/RudolphtheRedInter*png
Found corresponding PNG file /content/gdrive/MyDrive/BaselineExperiments/BaselineDataset/SheetImages/PNGFiles/RudolphtheRedInter.png
file is /content/gdrive/MyDrive/BaselineExperiments/BaselineDataset/SheetImages/PNGFiles/RudolphtheRedInter.png
2022-03-11 14:01:33 Dewarping
2022-03-11 14:01:41 Extracting stafflines
26 213 14
214 401 14
402 589 14
590 777 12
778 965 12
966 1153 12
1154 1341 12
1342 1529 12
2022-03-11 14:01:43 Extracting noteheads
2022-03-11 14:01:43 Analyzing notehead bboxes
2022-03-11 14:01:45 Instanitiating notes
2022-03-11 14:01:47 Grouping noteheads
2022-03-11 14:01:56 Extracting symbols
2022-03-11 14:01:57 Extracting rhythm types
2022-03-11 14:02:02 Building MusicXML document
Search string for pn

musicxml.xmlToM21: WARNING: Cannot put in an element with a missing voice tag when no previous voice tag was given.  Assuming voice 1... 


Accuracy for song JinglebellsInter.wav is 88.36%
Processing DanceofthePro.wav...
Skipping the transcription for file DanceofthePro.wav...
Search string for png file is /content/gdrive/MyDrive/BaselineExperiments/BaselineDataset/*/*/DanceofthePro*png
Found corresponding PNG file /content/gdrive/MyDrive/BaselineExperiments/BaselineDataset/SheetImages/PNGFiles/DanceofthePro.png
file is /content/gdrive/MyDrive/BaselineExperiments/BaselineDataset/SheetImages/PNGFiles/DanceofthePro.png
2022-03-11 14:04:26 Dewarping
2022-03-11 14:04:33 Extracting stafflines
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
26 206 9
207 387 10
388 568 10
569 749 10
750 930 10
931 1111 10
1112 1292 4
1293 1473 2
2022-03-11 14:04:34 Extracting noteheads
2022-03-11 14:04:34 

musicxml.xmlToM21: WARNING: Cannot put in an element with a missing voice tag when no previous voice tag was given.  Assuming voice 1... 
musicxml.xmlToM21: WARNING: Cannot put in an element with a missing voice tag when no previous voice tag was given.  Assuming voice 1... 
musicxml.xmlToM21: WARNING: Cannot put in an element with a missing voice tag when no previous voice tag was given.  Assuming voice 1... 
musicxml.xmlToM21: WARNING: Cannot put in an element with a missing voice tag when no previous voice tag was given.  Assuming voice 1... 
musicxml.xmlToM21: WARNING: Cannot put in an element with a missing voice tag when no previous voice tag was given.  Assuming voice 1... 
musicxml.xmlToM21: WARNING: Cannot put in an element with a missing voice tag when no previous voice tag was given.  Assuming voice 1... 
musicxml.xmlToM21: WARNING: Cannot put in an element with a missing voice tag when no previous voice tag was given.  Assuming voice 1... 
musicxml.xmlToM21: WARNING: Cannot

Accuracy for song JesuJoyPro.wav is 27.68%
Mean accuracy is 61.92%
